In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", module='seaborn')

In [2]:
with open('../mr-eve-mr.header') as f:
    cols = f.read().split(',')

edges = pd.read_csv('../mr-eve-mr.csv', header=None)
edges.columns = cols

non_zeros = edges.loc[edges['pval'] != 0]
zeros = edges.loc[edges['pval'] == 0]

del edges

zeros

,head_node,tail_node,method,nsnp,b,se,ci_low,ci_upp,pval,selection,moescore,source\n
141,ukb-b-20172,ebi-a-GCST005215,FE IVW,2,-0.458536,0.005123,-0.468577,-0.448494,0.0,DF,1.0,MR-EvE-2021-03-10
439,ukb-b-20172,ieu-a-761,FE IVW,2,-0.458536,0.005123,-0.468577,-0.448494,0.0,DF,1.0,MR-EvE-2021-03-10
614,ukb-b-20172,met-a-393,FE IVW,2,0.052300,0.000014,0.052272,0.052327,0.0,DF,1.0,MR-EvE-2021-03-10
817,ukb-b-20172,met-a-596,FE IVW,2,0.071661,0.001676,0.068377,0.074945,0.0,DF,1.0,MR-EvE-2021-03-10
919,ukb-b-20172,met-a-698,FE IVW,2,-0.120561,0.000727,-0.121985,-0.119137,0.0,DF,1.0,MR-EvE-2021-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...
25697170,prot-a-439,ukb-b-7675,FE IVW,2,0.000583,0.000014,0.000556,0.000610,0.0,DF,1.0,MR-EvE-2021-03-10
25697365,prot-a-439,ukb-b-9058,FE IVW,2,0.000387,0.000004,0.000379,0.000396,0.0,DF,1.0,MR-EvE-2021-03-10
25697370,prot-a-439,ukb-b-9125,FE IVW,2,0.000356,0.000009,0.000339,0.000374,0.0,DF,1.0,MR-EvE-2021-03-10
25697904,prot-a-439,ukb-d-30720_raw,FE IVW,3,0.001538,0.000013,0.001513,0.001564,0.0,DF,1.0,MR-EvE-2021-03-10


In [ ]:
lowest_pval = min(non_zeros['pval'])
print(f'Lowest non-zero pval = {lowest_pval}')

In [ ]:
sns.distplot(non_zeros['b:float'], rug=True)
plt.title('Distribution of effect size for edges with pval > 0')
plt.show()

In [ ]:
sns.distplot(zeros['b:float'], rug=True)
plt.title('Distribution of effect size for edges with pval=0')
plt.show()

In [ ]:
zeros.sort_values('b:float', ascending=False)

In [ ]:
sns.distplot(zeros.loc[np.abs(zeros['b:float']) < 100]['b:float'], norm_hist=False, kde=False)
plt.show()

In [ ]:
sns.distplot(zeros.loc[np.abs(zeros['b:float']) < 25]['b:float'], norm_hist=False, kde=False)
plt.show()

In [ ]:
sns.distplot(zeros.loc[np.abs(zeros['b:float']) < 2.5]['b:float'], norm_hist=False, kde=False)
plt.show()

In [ ]:
lim = 1
between_prop = round(sum(np.abs(zeros['b:float']) < lim)/len(zeros), 3)
print(f'{between_prop*100}% of edges exist between effect sizes {-lim} and {lim}')

In [ ]:
lim = 2.5
between_prop = round(sum(np.abs(zeros['b:float']) < lim)/len(zeros), 3)
print(f'{between_prop*100}% of edges exist between effect sizes {-lim} and {lim}')

In [ ]:
def null_overlap(args, null_val=0):
    a = args[0]
    b = args[1]
    if (a < null_val and b > null_val) or (a > null_val and b < null_val):
        return True
    return False

zeros['null'] = list(map(null_overlap, [[row['ci_low'], row['ci_upp']] for i, row in zeros.iterrows()]))

In [ ]:
print(f'Only {len(zeros.loc[zeros.null])} edges have CIs that cross the null\n')

zeros.loc[zeros.null]

# Check for synonymous traits

In [ ]:
attributes = pd.read_csv('../../../processed/mr-eve/attributes/node_attributes.csv')
attributes.head()

In [ ]:
synon_check_df = zeros[['head_node', 'tail_node']].merge(attributes[['id', 'trait']], how='left', left_on='head_node', right_on='id').merge(attributes[['id', 'trait']], how='left', left_on='tail_node', right_on='id').dropna(how='any', axis=0).reset_index(drop=True).drop(['id_x', 'id_y'], axis=1)

synon_check_df.head()

In [ ]:
sample = np.random.choice(synon_check_df.index, 100)
synon_edges = []
non_synon_edges = []
for ind in sample:
    row = synon_check_df.loc[ind]
    print(f'\n\n\nEdge: {row.head_node} - {row.tail_node}')
    print(f'\n {row.trait_x}')
    print(f'\n {row.trait_y}')
    while True:
        response = input('Is synonymous?')
        if response == 'y':
            synon_edges.append(row)
            break
        elif response == 'n':
            non_synon_edges.append(row)
            break
        elif response == 'skip':
            break

In [ ]:
print(f'{len(synon_edges)} of {len(synon_edges) + len(non_synon_edges)} edges from the sample were dememed synonymous:\n\n')

for edge in synon_edges:
    print(edge)
    print('\n')

# Check p-values by deriving Z scores

In [50]:
from scipy.stats import norm

In [55]:
zeros['z'] = zeros.b/zeros.se
zeros['p_derived'] = norm.sf(np.abs(zeros.z)) * 2 # double because 2-tailed test
zeros

,head_node,tail_node,method,nsnp,b,se,ci_low,ci_upp,pval,selection,moescore,source\n,z,p_derived
141,ukb-b-20172,ebi-a-GCST005215,FE IVW,2,-0.458536,0.005123,-0.468577,-0.448494,0.0,DF,1.0,MR-EvE-2021-03-10,-89.500831,0.0
439,ukb-b-20172,ieu-a-761,FE IVW,2,-0.458536,0.005123,-0.468577,-0.448494,0.0,DF,1.0,MR-EvE-2021-03-10,-89.500831,0.0
614,ukb-b-20172,met-a-393,FE IVW,2,0.052300,0.000014,0.052272,0.052327,0.0,DF,1.0,MR-EvE-2021-03-10,3730.091220,0.0
817,ukb-b-20172,met-a-596,FE IVW,2,0.071661,0.001676,0.068377,0.074945,0.0,DF,1.0,MR-EvE-2021-03-10,42.768744,0.0
919,ukb-b-20172,met-a-698,FE IVW,2,-0.120561,0.000727,-0.121985,-0.119137,0.0,DF,1.0,MR-EvE-2021-03-10,-165.947298,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25697170,prot-a-439,ukb-b-7675,FE IVW,2,0.000583,0.000014,0.000556,0.000610,0.0,DF,1.0,MR-EvE-2021-03-10,42.081577,0.0
25697365,prot-a-439,ukb-b-9058,FE IVW,2,0.000387,0.000004,0.000379,0.000396,0.0,DF,1.0,MR-EvE-2021-03-10,88.372093,0.0
25697370,prot-a-439,ukb-b-9125,FE IVW,2,0.000356,0.000009,0.000339,0.000374,0.0,DF,1.0,MR-EvE-2021-03-10,40.004332,0.0
25697904,prot-a-439,ukb-d-30720_raw,FE IVW,3,0.001538,0.000013,0.001513,0.001564,0.0,DF,1.0,MR-EvE-2021-03-10,119.324644,0.0


In [52]:
zeros.loc[zeros.p_derived > 0].sort_values('p_derived')

,head_node,tail_node,method,nsnp,b,se,ci_low,ci_upp,pval,selection,moescore,source\n,z,p_derived
3242922,met-c-841,ukb-a-344,FE IVW,2,0.101516,0.002694,0.096235,0.106797,0.0,DF,1.0,MR-EvE-2021-03-10,37.675814,1.236642e-310
8113645,prot-a-884,ubm-a-1260,FE IVW,2,0.076234,0.002023,0.072268,0.080199,0.0,DF,1.0,MR-EvE-2021-03-10,37.674888,1.280569e-310
14307887,prot-a-1403,ubm-a-1010,FE IVW,2,-0.019972,0.000530,-0.021011,-0.018933,0.0,DF,1.0,MR-EvE-2021-03-10,-37.674203,1.314066e-310
4299812,prot-a-607,ukb-b-20414,FE IVW,2,0.000354,0.000009,0.000335,0.000372,0.0,DF,1.0,MR-EvE-2021-03-10,37.674004,1.323957e-310
9349898,prot-a-10,ukb-a-289,FE IVW,2,-0.007796,0.000207,-0.008201,-0.007390,0.0,DF,1.0,MR-EvE-2021-03-10,-37.673428,1.353012e-310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24267995,prot-a-1174,ukb-b-5309,FE IVW,2,-0.043904,0.001170,-0.046197,-0.041611,0.0,DF,1.0,MR-EvE-2021-03-10,-37.525530,3.532383e-308
7409045,ubm-a-805,ubm-a-506,FE IVW,2,0.794422,0.021170,0.752929,0.835916,0.0,DF,1.0,MR-EvE-2021-03-10,37.525504,3.535929e-308
268402,prot-a-1573,ukb-b-1226,FE IVW,2,-0.036664,0.000977,-0.038579,-0.034749,0.0,DF,1.0,MR-EvE-2021-03-10,-37.523867,3.760078e-308
2865598,ukb-b-4943,prot-a-914,FE IVW,2,0.324615,0.008651,0.307658,0.341572,0.0,DF,1.0,MR-EvE-2021-03-10,37.521611,4.092387e-308


### The vast majority of derived p-values are also 0, with only 253 out of ~48000 being non-zero. Furthermore, of these 253 non-zeros the largest pval is 4.1e-308. This should act as strong evidence (pun intended) that winsorising these edge weights to the otherwise-highest value is a fine way of dealing with them.

# Checking CIs against standard errors

In [54]:
# Check non-zero-pval edges for funky SE and CIs
problem_ids = []
tol = 0.01
sample = np.random.choice(non_zeros.index, 1000)
for i in sample:
    row = non_zeros.loc[i]
    ci_low_diff = row.ci_low - (row.b - 1.96 * row.se)
    ci_upp_diff = row.ci_upp - (row.b + 1.96 * row.se)
    if np.abs(ci_low_diff) > tol or np.abs(ci_upp_diff) > tol:
        problem_ids.append(i)

non_zeros.loc[problem_ids]

,head_node,tail_node,method,nsnp,b,se,ci_low,ci_upp,pval,selection,moescore,source\n
23113877,prot-a-1306,prot-a-1052,FE IVW,4,0.022059,0.031179,-0.057397,0.101514,0.479267,DF,1.00,MR-EvE-2021-03-10
22332613,ieu-a-800,ubm-a-218,FE IVW,4,-0.030352,0.055652,-0.166503,0.105800,0.585492,DF,1.00,MR-EvE-2021-03-10
20643541,prot-a-2926,ubm-a-2040,FE IVW,3,-0.002798,0.025585,-0.076729,0.071132,0.912905,DF,1.00,MR-EvE-2021-03-10
12769650,ukb-a-313,ubm-a-1201,FE IVW,2,0.500982,0.518944,-0.920429,1.922394,0.334351,DF,1.00,MR-EvE-2021-03-10
16120012,ukb-a-26,ubm-a-49,FE IVW,13,0.091010,0.231631,-0.614371,0.796391,0.694386,Tophits,0.69,MR-EvE-2021-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...
18556491,ubm-a-146,ukb-b-2729,FE IVW,4,-0.058985,0.090764,-0.367170,0.249200,0.515774,DF,1.00,MR-EvE-2021-03-10
10688170,ubm-a-488,prot-a-3232,FE IVW,5,-0.061195,0.095849,-0.462611,0.340222,0.523181,DF,1.00,MR-EvE-2021-03-10
8588681,prot-a-1622,prot-a-20,FE IVW,4,0.012349,0.036057,-0.132330,0.157028,0.731990,DF,1.00,MR-EvE-2021-03-10
846459,prot-a-1930,prot-a-1340,FE IVW,4,0.048883,0.065208,-0.167158,0.264923,0.453466,DF,1.00,MR-EvE-2021-03-10


In [53]:
# Check zero-pval edges for funky SE and CIs

problem_ids = []
tol = 0.01
sample = np.random.choice(zeros.index, 1000)
for i in sample:
    row = zeros.loc[i]
    ci_low_diff = row.ci_low - (row.b - 1.96 * row.se)
    ci_upp_diff = row.ci_upp - (row.b + 1.96 * row.se)
    if np.abs(ci_low_diff) > tol or np.abs(ci_upp_diff) > tol:
        problem_ids.append(i)

zeros.loc[problem_ids]

,head_node,tail_node,method,nsnp,b,se,ci_low,ci_upp,pval,selection,moescore,source\n,z,p_derived
15999462,ukb-a-336,ukb-b-11842,FE IVW,208,0.440798,0.009320,0.378397,0.503199,0.0,Tophits,0.82,MR-EvE-2021-03-10,47.293490,0.0
3971976,ukb-b-14713,ukb-a-195,FE IVW,255,0.880678,0.011190,0.810222,0.951134,0.0,Tophits,0.78,MR-EvE-2021-03-10,78.702189,0.0
13385062,ukb-b-11842,ukb-b-2122,FE IVW,476,0.712920,0.010569,0.681097,0.744743,0.0,Tophits,0.87,MR-EvE-2021-03-10,67.453128,0.0
5696715,ieu-a-55,ukb-a-293,FE IVW,64,0.384501,0.009705,0.323840,0.445161,0.0,DF,0.91,MR-EvE-2021-03-10,39.620666,0.0
12630562,ukb-a-267,ukb-b-10787,FE IVW,367,0.973638,0.008045,0.912156,1.035119,0.0,Tophits,0.82,MR-EvE-2021-03-10,121.018601,0.0
7505440,prot-a-1854,ebi-a-GCST007236,FE IVW,4,1.040172,0.026286,0.094191,1.986152,0.0,DF,1.00,MR-EvE-2021-03-10,39.571596,0.0
7505440,prot-a-1854,ebi-a-GCST007236,FE IVW,4,1.040172,0.026286,0.094191,1.986152,0.0,DF,1.00,MR-EvE-2021-03-10,39.571596,0.0


### In samples of 1000, edges with 0-pvals actually have fewer cases where CI is not consistent with their SEs, compared to the non-0-pval edges.